In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import model_selection
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
# import lightgbm as lgb

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

In [ ]:
train_df = pd.read_csv('../input/quora-insincere-questions-classification/train.csv')
test_df = pd.read_csv("../input/quora-insincere-questions-classification/test.csv")
train_df.head()


In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

vect = TfidfVectorizer()
sklearn_tokenizer = vect.build_tokenizer()

vect = TfidfVectorizer(tokenizer= sklearn_tokenizer,stop_words='english')
clf = SGDClassifier(loss = 'log',class_weight = 'balanced')
model = Pipeline([('vect', vect), ('clf', clf)])
pipe = Pipeline([('vect', vect), ('clf', clf)])


In [ ]:
train_text = train_df['question_text']
test_text = test_df['question_text']
y_train=train_df['target']


In [ ]:
pipe.fit(train_text,y_train)

In [ ]:
from sklearn.model_selection import cross_val_predict, cross_val_score, \
    GridSearchCV, RandomizedSearchCV, train_test_split, KFold
from sklearn.linear_model import SGDRegressor

# Creating X, y Variables


all_text = train_text.append(test_text)
#tfidf_vectorizer = TfidfVectorizer()
#tfidf_vectorizer.fit(all_text)

grid = {
       # 'vect__max_features':[100, 2000],
       # 'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
        'vect__min_df' : [0.015, 0.035],# отбрасывает все слова, которые имеют повторяемость ниже чем x
        
      # 'vect_lowercase': [True, False],# переводит все заглавные буквы из слов из Заглавных букв в прописные
       'vect__stop_words':[{'english'},None],# определение словаря стоп-слов, если None, то словарь определяется в процессе обучения
        'clf__loss' : ['hinge','squared_hinge','perceptron'], # функция потерь, так как используем метрику roc_auc то нужна функция, которая выдаёт вероятностные знач принадлежности к классу
        'clf__alpha' : [0.005,0.0005], #параметр регуляризации (1/lambda)=c чем меньше тем сильнее регуляризация
       
        }

search = RandomizedSearchCV(pipe, param_distributions=grid, cv = 5,scoring='f1', verbose =1, n_jobs = -1)
search.fit(train_text,y_train)

In [ ]:
from sklearn import metrics
pipe_2=search.estimator
pipe_2.fit(train_text,y_train)
def print_report(pipe):
    
    y_pred = pipe.predict(train_df.question_text.values)
    report = metrics.classification_report(y_train, y_pred
       )
    print(report)
    print("accuracy: {:0.3f}".format(metrics.accuracy_score(y_train, y_pred)))
    print("rog_auc: {:0.3f}".format(metrics.roc_auc_score(y_train,y_pred)))
    print("f1: {:0.3f}".format(metrics.f1_score(y_train,y_pred)))
print_report(pipe_2)

In [ ]:
y_pred = pipe_2.predict(test_df.question_text.values);
y_pred

In [ ]:
train_text_features_cv = tfidf_vectorizer.transform(train_text)
test_text_features_cv = tfidf_vectorizer.transform(test_text)


In [ ]:
from nltk.corpus import stopwords

In [ ]:
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [ ]:
train_text = train_df['question_text']
test_text = test_df['question_text']
train_target = train_df['target']
all_text = train_text.append(test_text)

In [ ]:
submission1 = pd.DataFrame.from_dict({'qid': test_df['qid']})
submission1['prediction'] = y_pred
submission1.to_csv('submission.csv', index=False)


In [ ]:
submission1['prediction']

In [ ]:
submission1 = pd.DataFrame.from_dict({'qid': test_df['qid']})

In [ ]:
submission1

In [ ]:
np.size(y_pred)